In [1]:
import os 
os.chdir("../")

In [25]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class ModelEvalusationConfig:
  root_dir: Path
  data_path: Path
  model_path: Path
  tokenizer_path: Path
  metric_file_name: Path

In [3]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [26]:
class configurationManager:
    def __init__(
        self,
        config_filepath = ConfigFilePath,
        params_filepath = paramsFilePath):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    def getEvaluationConfig(self) -> ModelEvalusationConfig:
        config = self.config.model_evaluation
        create_directories([config.root_dir])
        ModelEValuationConfig=ModelEvalusationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name
        )
        return ModelEValuationConfig



In [27]:
config=configurationManager()
ModelEvaluationConfig=config.getEvaluationConfig()

[2024-09-25 13:28:28,239; INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-25 13:28:28,245; INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-25 13:28:28,248; INFO: common: created directory at: artifacts]
[2024-09-25 13:28:28,252; INFO: common: created directory at: artifacts/model_evaluation]


In [5]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from datasets import load_dataset,load_from_disk
import torch
import pandas as pd 
from tqdm import tqdm
from evaluate import load


c:\Users\PCS\Desktop\NLP end-to-end Project\endToend-NLP-Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-09-25 12:06:31,976; INFO: config: PyTorch version 2.4.1 available.]


In [38]:
class ModelEvaluation:
    def __init__(self,config:ModelEvalusationConfig) -> None:
        self.config=config
    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]



    def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer, 
                                batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
                                column_text="article", 
                                column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                            padding="max_length", return_tensors="pt")
            
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device), 
                            length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
            
            # Finally, we decode the generated texts, 
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=True) 
                for s in summaries]      
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            
            metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score
    def evaluate(self):
        device="cuda" if torch.cuda.is_available() else "cpu"
        tokenizer=AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        dataset_samsum_pt=load_from_disk(self.config.data_path)
        
        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        rouge_metric = load('rouge')
        print(dataset_samsum_pt['test'][0:10])
        score=self.calculate_metric_on_test_ds(dataset_samsum_pt['test'][0:10],rouge_metric,model_pegasus,tokenizer,batch_size=2,device=device,column_text="dialogue",column_summary="summary")
        rouge_dict=dict((rn,score[rn].mid.fmeasure) for rn  in rouge_names)

        df=pd.DataFrame(rouge_dict,index=['peagasus'])
        df.to_csv(self.config.metric_file_name,index=False)


In [12]:
t=ModelEvaluation(config)

In [39]:
try:
    config=configurationManager()
    ModelEvaluationConfig=config.getEvaluationConfig()
    ModelEvaluation=ModelEvaluation(ModelEvaluationConfig)
    ModelEvaluation.evaluate()
except Exception as e :
    raise e

[2024-09-25 13:36:47,855; INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-25 13:36:47,858; INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-25 13:36:47,860; INFO: common: created directory at: artifacts]
[2024-09-25 13:36:47,862; INFO: common: created directory at: artifacts/model_evaluation]
{'id': ['13862856', '13729565', '13680171', '13729438', '13828600', '13716964', '13731487', '13814882', '13680876', '13809974'], 'dialogue': ["Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye", "Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric: I know! And shows how Americans see Russian ;)\

  0%|          | 0/5 [00:26<?, ?it/s]


KeyboardInterrupt: 